## Imports

In [1]:
import math
import pandas as pd
import plotly.graph_objects as go
import requests
from datetime import datetime as dt
from pathlib import Path

## Functions

In [2]:
# Poleni
def Q_poleni(mu, L, h_w):
    return 2/3 * mu * math.sqrt(2*9.81) * L * h_w**(3/2)

# Get data from Messdatenbank
def get_data(
    hbt_id: int | str,
    dt_from: str | dt,
    dt_to: str | dt,
    dt_format: str = '%d.%m.%Y %H:%M:%S',
    limit: int | str = 0,
    ) -> pd.DataFrame:

    # Global parameters
    server = 'http://dataserver.hunziker-betatech.ch:5000'
    endpoint: str = '/messdaten/API/v1.0/'

    # Set dates
    if isinstance(dt_from, str):
        dt_from = dt.strptime(dt_from, dt_format)

    if isinstance(dt_to, str):
        dt_to = dt.strptime(dt_to, dt_format)
    
    dt_from = dt_from.isoformat()
    dt_to = dt_to.isoformat()

    # Set ID, limit
    hbt_id = str(hbt_id)
    limit = str(limit)

    # Set params
    params = {
        'format': 'json',
        'time_from': dt_from,
        'time_to': dt_to,
        'limit': limit
    }

    # Build URL
    url = f"{server}{endpoint}messeinheit/{hbt_id}/messresultat"

    # Get the data
    r = requests.get(url, params=params)

    # Convert from JSON
    data = r.json()

    # Read data to DataFrame
    df = pd.DataFrame(data['messresultate'])

    # Set 'zeit_phaenomen' as index and remove timezone
    df['datum'] = pd.to_dt(df['zeit_phaenomen'].apply(str), 
        format="%Y-%m-%dT%H:%M") 
    df['datum'] = df['datum'].apply(lambda dt: dt.replace(tzinfo=None))
    df.set_index('datum', inplace=True)

    # Remove unnecessary columns
    df.drop(columns=['zeit_phaenomen', 'id', 'qualitaet'], inplace=True)

    return df

## Settings

In [3]:
h_nws = 430.70 # m ü.M.
h_hws = 431.05 # m ü.M.
L_ueb = 1.9 # m

save_dir = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\SABA")

## Read data

In [4]:
dt_format = '%d.%m.%Y %H:%M:%S'
dt_start = dt.strptime("21.04.2022 12:00:00", dt_format)
dt_end = dt.strptime("01.09.2022 00:00:00", dt_format)
h_asb = get_data(919, dt_from=dt_start, dt_to=dt_end)
h_0 = h_asb.iloc[:20].mean().values[0]
h_asb += (h_nws - h_0)

## Analysis

In [5]:
df_ueb = h_asb[h_asb > h_hws]
df_ueb.dropna(inplace=True)
df_ueb -= h_hws
df_ueb['Q [l/s]'] = Q_poleni(0.6, L_ueb, df_ueb['wert']) * 1e3
df_ueb['Vol [l]'] = df_ueb['Q [l/s]'] * 60
df_ueb['Vol_cum [m3]'] = df_ueb['Vol [l]'].cumsum() / 1e3

save_path = save_dir / 'Überlaufmengen_ASB.csv'
df_ueb.to_csv(save_path, sep=';')

## Plots

In [10]:
fig = go.Figure()

x = h_asb.index
y = h_asb.iloc[:,0]
fig.add_trace(go.Scatter(x=x, y=y))
fig.add_hline(y=h_hws, line_width=2, line_color='red')
fig.add_annotation(x=x[0], y=h_hws,text=f'{h_hws} m ü.M.', font=dict(size=25,
    color='red'))
fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Niveau [m ü.M.]')
fig.update_layout(title='Niveau Absetzbecken', font=dict(size=25))

save_path = save_dir / 'Niveau_ASB.html'
fig.write_html(save_path)